  <picture>
  <source srcset="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes-w.svg" media="(prefers-color-scheme: dark)">
    <img src="https://raw.githubusercontent.com/Archfx/RACKETutes/main/images/racketutes.svg">
  </picture>

Rosette101
===

The previous post was about getting familiar with Racket. To make things more interesting we have used Jupyter notebook interactive environment. This time was are going to use Rosette in the same environment.

Rosette is a solver-aided programming language. To be specific popular [Z3](https://github.com/Z3Prover/z3) is behind the curtains of Rosette. With the power of Z3, Rosette programs can understand and handle symbolic values, assertions, assumptions, and queries. Which is a vital part of verifying firmware and hardware designs.

Note: Rosette's symbolic evaluation engine operates on the program states. Instead of operating on one path at a time, it keeps track of all assumptions and assertions in the path at the beginning.  This is done through the `verification` condition (VC)`.  During the tutorial, we will be switching contexts between different examples and problems. Since we are using the same kernel to run different examples, we need to clean the VC when we switch the context.

In [1]:
#lang iracket/lang #:require rosette/safe 

Symbolic Values
---

When it comes to validating designs and implementations, one thing is we can't visualize all the internal variables. These variables are sometimes constrained by different conditions. With Rosette, we can keep these variables as unknown variable variables which refers to the technical term of ```symbolic variables```. These variables are symbolically evaluated on the constraints that act on them. 

In [2]:
(define-symbolic b boolean?)
(displayln b)

(displayln (boolean? b))
(displayln (integer? b))
(displayln (vector b 1))
(displayln (not b))
(displayln (boolean? (not b)))

b
#t
#f
#(b 1)
(! b)
#t


Assertions and Assumptions
---

As the name suggests, Rosette's assertions work exactly similarly to assertions used in other languages. If the assertion is failed, it will terminate the exection with an exception.

In [21]:
(assert #t)
(assert #f)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level


In [22]:
(clear-vc!)

In [23]:
(define-symbolic i j integer?)
(assume (> j 0))     ; Add the assumption (> j 0) to the VC.
(vc-assumes (vc))

(< 0 j)

In [24]:
(assert (< (- i j) i))
(vc-asserts (vc))    ; The assertions must hold when the assumptions hold.

(display (vc))

#(struct:vc (< 0 j) (|| (! (< 0 j)) (< (+ i (- j)) i)))

In [6]:
; int32? is a shorthand for the type (bitvector 32).
(define int32? (bitvector 32))

; int32 takes as input an integer literal and returns
; the corresponding 32-bit bitvector value.
(define (int32 i)
  (bv i int32?))


In [ ]:
(clear-vc!)

Solver-Aided Queries
---

In [7]:
(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x 1) (+ x 2) (+ x 2)))
 
(define (same p f x)
 (assert (= (p x) (f x))))

In [10]:
(same poly factored 0)
(same poly factored -1)
(same poly factored -2)

In [ ]:
(clear-vc!)

Verification
---

Program verification solves the problem of verifying the program for all the legal inputs. The brute force approach would be to iterate through all the values one by one. Although this sounds promising at capturing any bug if exists, in the case of a 32-bit adder circuit, input space would be $2^64$ which is a large number.
Instead of going through the brute force path, Rosette delegates this task to `z3 constraint solver` with the help of `verify` query.


In [11]:
(define-symbolic i integer?)
(define cex (verify (same poly factored i)))


In [15]:
(evaluate i cex)
(poly -6)
(factored -6)
; (same poly factored -6)

480

In [25]:
(clear-vc!)


Synthesis
----

In [26]:
(require rosette/lib/synthax)

(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* (+ x (??)) (+ x 1) (+ x (??)) (+ x (??))))
 
(define (same p f x)
 (assert (= (p x) (f x))))


In [ ]:
; Call after saving the above definitions to a file:
(define-symbolic i integer?)

(define binding
    (synthesize #:forall (list i)
                #:guarantee (same poly factored i)))
(print-forms binding)
'(define (factored x) (* (+ x 0) (+ x 1) (+ x 2) (+ x 3)))



In [ ]:
(clear-vc!)

Angelic Execution
----

In [18]:
(define-symbolic x y integer?)
(define sol
    (solve (begin (assert (not (= x y)))
                  (assert (< (abs x) 10))
                  (assert (< (abs y) 10))
                  (assert (not (= (poly x) 0)))
                  (assert (= (poly x) (poly y))))))
(evaluate x sol)

(evaluate y sol)

(evaluate (poly x) sol)

(evaluate (poly y) sol)

application: not a procedure;
 expected a procedure that can be applied to arguments
  given: (unsat)
  context...:
   body of top-level
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/query/eval.rkt:19:0: eval-rec
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [ ]:
(clear-vc!)

### Some Problems from Internet


Finding an integer whose absolute value is X with Racket

In [1]:
#lang iracket/lang #:require rosette/safe 

(define (abs x)
    (if (< x 0) (- x) x))

In [2]:
; define a symbolic variable y

(define-symbolic y integer?)

In [3]:
; Solve a constraint saying |y| = 5.
(solve
  (assert (= (abs y) 5)))

(model
 [y -5])

Now let us look at some sample questions from CAV 2019 which was developed by [Emina](https://github.com/emina)




### Q1. Warmup

Suppose that you are an app developer for a super low-power chip that achieves peak energy efficiency on programs with few or no branches. To build apps faster, you hire Ben Bitdiddle to create a library of efficient primitives for this chip, starting with a branch-free function that computes the absolute value of a 32-bit bitvector. 

In [1]:
#lang iracket/lang #:require rosette

In [2]:
(define (abs-spec x)
  (if (bvslt x (bv 0 32))
      (bvneg x)
      x))

Ben comes up with the following implementation that works on his test suite:

In [3]:
(define (abs-impl x) 
  (let* ([o1 (bvashr x (bv 31 32))]
         [o2 (bvadd x o1)]
         [o3 (bvsub o2 o1)])
    o3))

; zero, positive, negative ...
(assert (equal? (abs-impl (bv #x00000000 32)) 
                  (abs-spec (bv #x00000000 32))))
(assert (equal? (abs-impl (bv #x00000003 32)) 
                  (abs-spec (bv #x00000003 32))))
(assert (equal? (abs-impl (bv #x80000000 32)) 
                  (abs-spec (bv #x80000000 32))))

Help review Ben’s code by answering the following questions. If a question requires you to modify his implementation in any way, do so on a fresh copy with a new name such as abs-impl-N where N is the question number.

Does Ben’s implementation work on all 32-bit inputs? Use the verify query to check.

In [5]:
; Let's define a generic 32bit vector
(define-symbolic x (bitvector 32))

; Using Rosette verify query
(define check
  (verify
   (assert(equal? (abs-spec x) (abs-impl x)))))

(evaluate check x)

(model
 [x (bv #x8404c7c0 32)])

Apparently, we have a counter-example where implementation does not satisfy the specification.

The verifier will reveal that Ben’s implementation is buggy. Use debug to localize the fault found by the verifier (and remember to save your work to a file before invoking debug).

Use synthesize to fix Ben’s implementation as suggested by the debugger. You can assume that if Ben has made an error, it is always of the same kind: using an arithmetic operator (such as addition or subtraction) instead of a bitwise operator that takes the same arguments. It is easiest to sketch this knowledge using the choose construct.

In [10]:
(require rosette/lib/synthax)

(define (abs-impl-3 x) 
  (let* ([o1 (bvashr x (bv 31 32))]
         [o2 ((choose bvadd bvand bvor bvxor bvshl bvlshr bvashr) x o1)]
         [o3 ((choose bvsub bvand bvor bvxor bvshl bvlshr bvashr) o2 o1)])
    o3))


In [12]:
(synthesize
  #:forall x
  #:guarantee (assert (equal? (abs-spec x) (abs-impl-3 x))))

(model
 [0$choose:eval:5:15 #t]
 [1$choose:eval:5:15 #f]
 [2$choose:eval:5:15 #f]
 [3$choose:eval:5:15 #f]
 [4$choose:eval:5:15 #f]
 [5$choose:eval:5:15 #f]
 [0$choose:eval:6:15 #f]
 [1$choose:eval:6:15 #f]
 [2$choose:eval:6:15 #f]
 [3$choose:eval:6:15 #t])

### Q2. Majority Voting Algorithm

The [Boyer–Moore majority vote algorithm](https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore_majority_vote_algorithm) is a beautiful and tricky procedure for finding a majority element in a list, using linear time and constant space.

Complete the following Rosette implementation of Boyer-Moore and verify its correctness for any list of N arbitrary integers.

In [1]:
#lang iracket/lang #:require rosette

In [2]:
(define (boyer-moore xs)
  (define m (car xs))  ; car returns the first element of an array
  (define i 0)
  (for ([x xs])
    (if (= i 0) (and (set! m x) (set! i 1))
          (if (= m x) (set! i (+ i 1)) 
          (set! i (- i 1)))
          ))
  m)

Test for several test cases in traditional way

In [3]:
(display "case 1 :")(displayln (= 1 (boyer-moore `(1 1 2 4 1 1))))
(display "case 2 :")(displayln (= 4 (boyer-moore `(8 6 4 1 2 4 1 1 8 4 8 4 4))))

case 1 :#t
case 2 :#t


In [4]:
;Check the algorithm for a list of n arbitrary integers.
(define (check n [bw #f])
  (current-bitwidth bw)
  (define-symbolic* xs integer? [n])
  (define-symbolic* m integer?)
  (verify 
  #:assume ; there is a majority value ...
   (assert (> (count (curry = m) xs)
              (quotient n 2)))
   #:guarantee ; algorithm works
   (assert (= m (boyer-moore xs)))))

(time (check 10)) 



eval:6:2: verify: use does not match pattern: (verify expr)
  in: (verify #:assume (assert (> (count (curry = m) xs) (quotient n 2))) #:guarantee (assert (= m (boyer-moore xs))))
  location...:
   eval:6:2
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/form/module.rkt:33:2: try-next
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop


In [5]:
; Check the algorithm for a list of n arbitrary integers.
(define (check2 n [bw #f])
  (current-bitwidth bw)
  (define-symbolic* xs integer? [n])
  (define-symbolic* m integer?)
  (verify
   (when ; there is a majority value ...
       (> (count (curry = m) xs)
          (quotient n 2))
     (assert (= m (boyer-moore xs)))))) ; algorithm works

(time (check2 1)) 


application: not a procedure;
 expected a procedure that can be applied to arguments
  given: 1
  context...:
   body of top-level
   eval:2:0: check2


Q3. Sudoku

In [57]:
#lang iracket/lang #:require rosette

In [59]:
(define (choice type) 
  (define-symbolic* c type) 
  c)

(define (more-or-less-1 x)
  (if (choice boolean?)
      (+ x 1)
      (- x 1)))

(solve (assert (= 0 (more-or-less-1 1)))) ; #f
(solve (assert (= 2 (more-or-less-1 1)))) ; #t

(model
 [a #t]
 [b #t]
 [c$6 #t])

In [60]:
(assert #f)

[assert] failed
  context...:
   /root/.local/share/racket/8.2/pkgs/rosette/rosette/base/core/exn.rkt:59:11: body of top-level


In [61]:
(define (poly x)
 (+ (* x x x x) (* 6 x x x) (* 11 x x) (* 6 x)))
 
(define (factored x)
 (* x (+ x 1) (+ x 2) (+ x 2)))
 
(define (same p f x)
 (assert (= (p x) (f x))))

In [66]:
(same poly factored 0)

In [63]:
(same poly factored -1)

In [64]:
(same poly factored -2)

In [2]:
#lang iracket/lang #:require rosette/safe 

(define-symbolic l h int32?)

(define cex (verify (check-mid bvmid l h)))


int32?: undefined;
 cannot reference an identifier before its definition
  in module: top-level
  context...:
   /usr/share/racket/pkgs/sandbox-lib/racket/sandbox.rkt:703:9: loop
